Набор данных, с информацией о ценообразующих факторах жилых помещений, расположенных в городе Омск, с указанием стоимости каждого объекта. В данном файле представлен разведовательный анализ данных, проведенный средствами google.colab

Импорт необходимых библиотек

In [ ]:
# Лучше больше, чем меньше

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.impute import SimpleImputer # Объект для замены пропущенных значений
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler # Импортируем нормализацию и One-Hot Encoding от scki-kit-learn
from sklearn.preprocessing import PowerTransformer
from sklearn.pipeline import Pipeline # Pipeline.Не добавить, не убавить
from google.colab import output
from sklearn.compose import ColumnTransformer # т.н. преобразователь колонок
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge # Регуляризация Ridge от scikit-learn

Создание необходимых функций

In [ ]:
def cat_num_split(df: pd.DataFrame) -> tuple:
    '''Ищем категориальные и числовые признаки в датафрейме'''

    cat_columns = []
    num_columns = []

    for column_name in df.columns:
        if (df[column_name].dtypes == object):
            cat_columns +=[column_name]
        else:
            num_columns +=[column_name]

    print('categorical columns:\t ',cat_columns, '\n len = ',len(cat_columns))
    print('numerical columns:\t ',  num_columns, '\n len = ',len(num_columns))

    return cat_columns, num_columns

Загружаем dateset

In [ ]:
from google.colab import files

In [ ]:
files.upload()

In [ ]:
# Датасет по недвижимости в г. Омске за 2022 г.
omsk = pd.read_excel('/content/Omsk_ed.xlsx')

Первый взгляд на dateset

In [ ]:
omsk.sample(5)

,UID,kadastr,material,mat2,build_year,year_group,type,floortype,floorrange,site,...,hish_center_path,ODC_path,underwater_path,way_path,highway_path,bigway_path,stoppoint_path,mail_path,factory_path,line
1186,1926_КВ_2_САО_ Волкова_43005,55:36:070105:3427,Крупнопанельные,КС3,1970.0,1961-1984,2,среднеэтажка,последний,https://omsk.cian.ru/cat.php?currency=2&deal_t...,...,5.742,1.303,3.073,0.138,2.481,17.421,0.378,0.863,0.538,Не первая
1601,3105_КВ_1_КАО_ проспект Комарова_55109,55:36:110106:416,Каменные,КС1,2007.0,более2005,1,многоэтажка,средний,https://omsk.cian.ru/cat.php?currency=2&deal_t...,...,3.081,1.122,2.948,0.179,5.780,12.086,0.258,0.382,0.976,Не первая
167,2400_КВ_1_ОАО_ 5-я Рабочая_46604,55:36:120307:13148,Кирпичные,КС1,1980.0,1961-1984,1,многоэтажка,средний,https://omsk.cian.ru/cat.php?currency=2&deal_t...,...,3.408,2.289,7.136,0.039,7.136,6.063,0.103,0.609,0.139,Не первая
56,184_КВ_1_ОАО_ 4-я Кордная_25043,55:36:120305:4250,Кирпичные,КС1,1957.0,до 1960,мс,малоэтажка,первый,https://omsk.cian.ru/cat.php?currency=2&deal_t...,...,4.181,0.731,5.803,0.039,6.123,6.952,0.235,1.000,0.111,Не первая
128,1112_КВ_1_САО_ 50 лет Профсоюзов_36869,55:36:050207:15489,Кирпичные,КС1,1963.0,1961-1984,1,среднеэтажка,средний,https://omsk.cian.ru/cat.php?currency=2&deal_t...,...,7.352,0.190,2.463,0.019,4.673,15.340,0.250,0.203,0.186,Не первая


In [ ]:
omsk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2805 entries, 0 to 2804
Data columns (total 60 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   UID               2805 non-null   object 
 1   kadastr           2805 non-null   object 
 2   material          2805 non-null   object 
 3   mat2              2805 non-null   object 
 4   build_year        2735 non-null   float64
 5   year_group        2805 non-null   object 
 6   type              2805 non-null   object 
 7   floortype         2805 non-null   object 
 8   floorrange        2805 non-null   object 
 9   site              2805 non-null   object 
 10  date              2805 non-null   object 
 11  region            2805 non-null   object 
 12  city              2805 non-null   object 
 13  area              2805 non-null   object 
 14  subarea           2805 non-null   object 
 15  street            2805 non-null   object 
 16  house             1847 non-null   object 


In [ ]:
# Разделяем категориальные и числовые параметры
cat_columns, num_columns = cat_num_split(omsk)

categorical columns:	  ['UID', 'kadastr', 'material', 'mat2', 'year_group', 'type', 'floortype', 'floorrange', 'site', 'date', 'region', 'city', 'area', 'subarea', 'street', 'house', 'test', 'company', 'phone', 'square_group', 'arr', 'house_', 'street_', 'line'] 
 len =  24
numerical columns:	  ['build_year', 'price', 'площадь', 'meter_price', 'floor_N', 'floor_max', 'koef', 'month', 'korr', 'price_', 'price__', 'skor', 'group_', 'group___', 'rooms', 'ud_price', 't_', 'Х', 'У', 'cent_path', 'water_path', 'lock_cent_path', 'shop_path', 'nedr_path', 'bad_place_path', 'railway_path', 'electr_line_path', 'hish_center_path', 'ODC_path', 'underwater_path', 'way_path', 'highway_path', 'bigway_path', 'stoppoint_path', 'mail_path', 'factory_path'] 
 len =  36


In [ ]:
# Делаем разные датафреймы с категориальными и числовыми параметрами
omsk_cat = omsk[cat_columns]
omsk_num = omsk[num_columns]

In [ ]:
omsk_cat.head(2)

,UID,kadastr,material,mat2,year_group,type,floortype,floorrange,site,date,...,street,house,test,company,phone,square_group,arr,house_,street_,line
0,1883_КВ_1_САО_ проспект Мира_42646,55:36:000000:21065,Кирпичные,КС1,1961-1984,1,среднеэтажка,последний,https://omsk.cian.ru/cat.php?currency=2&deal_t...,44192,...,проспект Мира,82/1,Продаётся квартира отличный вариант для студен...,Этажи Омск,+7 934 335-91-96,до 30,САО,82/1,проспект Мира,Первая
1,1615_КВ_1_САО_ Бородина_40758,55:36:070102:10125,Кирпичные,КС1,1961-1984,1,среднеэтажка,средний,https://omsk.cian.ru/cat.php?currency=2&deal_t...,44191,...,улица Бородина,44Б,Светлая и уютная квартира студийного типа с со...,Собственник,+7 986 800-46-31,до 30,САО,44Б,Бородина,Не первая


In [ ]:
omsk_num.head(2)

,build_year,price,площадь,meter_price,floor_N,floor_max,koef,month,korr,price_,...,electr_line_path,hish_center_path,ODC_path,underwater_path,way_path,highway_path,bigway_path,stoppoint_path,mail_path,factory_path
0,1966.0,700000,13.0,53846.153846,5,5,17.5,24,700000.0,672000.0,...,0.059,7.171,0.520,1.484,0.032,5.203,15.285,0.098,0.677,0.437
1,1974.0,669000,13.0,51461.538462,2,5,17.5,24,669000.0,642240.0,...,0.012,6.504,3.169,2.737,0.130,2.001,18.046,0.166,0.426,0.010


Таблица корреляции

In [ ]:
omsk_num = omsk_num.dropna()

In [ ]:
corr = omsk_num.corr()

In [ ]:
corr.style.background_gradient(cmap='RdYlGn')

,build_year,price,площадь,meter_price,floor_N,floor_max,koef,month,korr,price_,price__,skor,group_,group___,rooms,ud_price,t_,Х,У,cent_path,water_path,lock_cent_path,shop_path,nedr_path,bad_place_path,railway_path,electr_line_path,hish_center_path,ODC_path,underwater_path,way_path,highway_path,bigway_path,stoppoint_path,mail_path,factory_path
build_year,1.000000,0.391809,0.142845,0.448487,0.385080,0.675091,-0.352224,0.062367,0.274301,0.391534,0.411485,0.469649,-0.358960,-0.358960,-0.179963,0.469649,0.457797,-0.016103,-0.105481,-0.053998,-0.019060,-0.061467,0.102660,0.032165,-0.323039,0.025921,0.219842,-0.055434,-0.000228,-0.105870,-0.066058,-0.164695,-0.044209,-0.045324,0.046416,-0.041920
price,0.391809,1.000000,0.760147,0.585710,0.228113,0.458195,-0.167037,0.076776,0.660044,0.999965,0.997766,0.581911,-0.184277,-0.184277,0.472211,0.581911,0.555531,-0.059288,-0.118188,-0.327959,-0.195738,-0.208139,0.335053,-0.085461,-0.187198,-0.088589,-0.075152,-0.327016,-0.258885,0.056189,-0.185954,-0.097644,0.030578,-0.178522,-0.288353,-0.153181
площадь,0.142845,0.760147,1.000000,-0.042162,0.041106,0.152904,0.002672,0.017183,0.424964,0.760598,0.749776,-0.042017,-0.002435,-0.002435,0.767755,-0.042017,-0.048737,-0.020506,-0.038990,-0.079912,-0.091177,-0.042113,0.131002,-0.035399,-0.051115,-0.006014,-0.011124,-0.081470,-0.054252,0.026057,-0.034604,0.004310,-0.008842,-0.037515,-0.035936,-0.029301
meter_price,0.448487,0.585710,-0.042162,1.000000,0.318963,0.533627,-0.291110,0.108915,0.476862,0.585177,0.595616,0.995524,-0.312515,-0.312515,-0.216527,0.995524,0.961819,-0.067740,-0.156619,-0.432625,-0.186346,-0.303742,0.354850,-0.067283,-0.209141,-0.155335,-0.122215,-0.430713,-0.361168,0.044910,-0.277937,-0.181773,0.051578,-0.262496,-0.420773,-0.230540
floor_N,0.385080,0.228113,0.041106,0.318963,1.000000,0.567818,-0.292062,0.025705,0.197597,0.228113,0.244301,0.337931,-0.298507,-0.298507,-0.089285,0.337931,0.327197,-0.041398,-0.017062,-0.072599,-0.069462,-0.039902,0.098339,-0.003763,-0.180804,0.040640,-0.012040,-0.072646,-0.043045,-0.082630,-0.032851,-0.050443,0.039073,-0.028472,-0.086966,-0.002582
floor_max,0.675091,0.458195,0.152904,0.533627,0.567818,1.000000,-0.456123,0.055468,0.353427,0.458064,0.483127,0.562296,-0.466545,-0.466545,-0.096943,0.562296,0.547181,-0.059063,-0.049284,-0.149825,-0.160342,-0.072623,0.214690,-0.017379,-0.304127,0.060696,0.006564,-0.151925,-0.093724,-0.101422,-0.058619,-0.071267,0.036803,-0.049653,-0.161549,-0.013035
koef,-0.352224,-0.167037,0.002672,-0.291110,-0.292062,-0.456123,1.000000,-0.066097,-0.100743,-0.166814,-0.225199,-0.375483,0.996566,0.996566,0.018369,-0.375483,-0.373084,0.024921,-0.037454,-0.007840,0.016592,-0.004051,-0.005070,-0.057855,0.067038,-0.059895,0.089759,-0.009265,0.001747,0.115049,-0.017873,0.027397,-0.060834,-0.018377,0.124448,-0.034018
month,0.062367,0.076776,0.017183,0.108915,0.025705,0.055468,-0.066097,1.000000,0.049195,0.069813,0.071780,0.099818,-0.068497,-0.068497,-0.010265,0.099818,0.105692,-0.027837,0.009086,-0.037266,0.056502,-0.036939,0.019700,0.019491,-0.013551,-0.015706,-0.010352,-0.036165,-0.036196,-0.014336,-0.033331,-0.043470,0.040892,-0.029489,-0.086828,-0.021568
korr,0.274301,0.660044,0.424964,0.476862,0.197597,0.353427,-0.100743,0.049195,1.000000,0.660017,0.663334,0.477798,-0.165001,-0.165001,0.242204,0.477798,0.462408,-0.427284,0.210712,-0.282218,-0.100371,-0.122715,0.269697,-0.294838,-0.349134,-0.029175,0.012909,-0.243655,-0.218380,-0.150428,-0.144801,-0.277594,0.496978,-0.142875,-0.291330,-0.115076
price_,0.391534,0.999965,0.760598,0.585177,0.228113,0.458064,-0.166814,0.069813,0.660017,1.000000,0.997791,0.581476,-0.184044,-0.184044,0.472614,0.581476,0.555020,-0.059038,-0.118466,-0.328091,-0.196282,-0.208139,0.335125,-0.085609,-0.187046,-0.088705,-0.075380,-0.327160,-0.259000,0.056502,-0.185956,-0.097404,0.030191,-0.178528,-0.288159,-0.153258


Вывод по таблице корреляции:

In [ ]:
# Предлагаю пока взять вот эти параметры, наиболее значимые по таблице корреляции. Если что, можно будет вернуться.

num_pred = ['price__', 'build_year', 'площадь', 'floor_N', 'floor_max', 'rooms', 't_', 'shop_path', 'cent_path', 'hish_center_path', 'mail_path'] #'korr', 'skor', 'ud_price',

In [ ]:
omsk_num = omsk_num[num_pred]

In [ ]:
omsk_num

,price__,build_year,площадь,floor_N,floor_max,rooms,t_,shop_path,cent_path,hish_center_path,mail_path
0,554400.000,1966.0,13.0,5,5,1,1883.0,5.347,8.506,7.171,0.677
1,529848.000,1974.0,13.0,2,5,1,1615.0,1.439,7.351,6.504,0.426
2,554400.000,1974.0,13.0,2,5,1,1884.0,1.439,7.351,6.504,0.426
3,640800.000,1983.0,16.0,2,4,1,1532.0,6.105,7.741,5.810,2.529
4,776160.000,2011.0,16.0,1,9,1,2601.0,7.551,5.491,3.980,0.272
...,...,...,...,...,...,...,...,...,...,...,...
2800,2016000.000,1979.0,131.0,2,2,3,16.0,2.598,42.831,42.114,2.980
2801,4824000.000,1991.0,140.9,2,4,3,776.0,8.852,9.951,8.443,0.626
2802,5032800.000,1991.0,147.0,1,4,3,775.0,8.852,9.951,8.443,0.626
2803,5176574.568,2004.0,150.0,2,8,2,801.0,8.233,2.180,0.139,0.156


In [ ]:
corr = omsk_num.corr()

In [ ]:
corr.style.background_gradient(cmap='RdYlGn')

,price__,build_year,площадь,floor_N,floor_max,rooms,t_,shop_path,cent_path,hish_center_path,mail_path
price__,1.000000,0.411485,0.749776,0.244301,0.483127,0.465087,0.571544,0.331188,-0.324302,-0.323118,-0.289409
build_year,0.411485,1.000000,0.142845,0.385080,0.675091,-0.179963,0.457797,0.102660,-0.053998,-0.055434,0.046416
площадь,0.749776,0.142845,1.000000,0.041106,0.152904,0.767755,-0.048737,0.131002,-0.079912,-0.081470,-0.035936
floor_N,0.244301,0.385080,0.041106,1.000000,0.567818,-0.089285,0.327197,0.098339,-0.072599,-0.072646,-0.086966
floor_max,0.483127,0.675091,0.152904,0.567818,1.000000,-0.096943,0.547181,0.214690,-0.149825,-0.151925,-0.161549
rooms,0.465087,-0.179963,0.767755,-0.089285,-0.096943,1.000000,-0.207144,0.009861,-0.009447,-0.009629,0.006587
t_,0.571544,0.457797,-0.048737,0.327197,0.547181,-0.207144,1.000000,0.343989,-0.371817,-0.368398,-0.404534
shop_path,0.331188,0.102660,0.131002,0.098339,0.214690,0.009861,0.343989,1.000000,-0.358908,-0.379435,-0.272281
cent_path,-0.324302,-0.053998,-0.079912,-0.072599,-0.149825,-0.009447,-0.371817,-0.358908,1.000000,0.995816,0.530941
hish_center_path,-0.323118,-0.055434,-0.081470,-0.072646,-0.151925,-0.009629,-0.368398,-0.379435,0.995816,1.000000,0.523262


Категориальные признаки

In [ ]:
cat_pred = ['material', 'year_group', 'type', 'floortype', 'floorrange','area', 'subarea', 'street', 'house']#, 'test','square_group', 'arr', 'house_', 'street_',]

In [ ]:
omsk[cat_pred]

,material,year_group,type,floortype,floorrange,area,subarea,street,house
0,Кирпичные,1961-1984,1,среднеэтажка,последний,Советский,мкр. Город Нефтяников,проспект Мира,82/1
1,Кирпичные,1961-1984,1,среднеэтажка,средний,Советский,мкр. Заозерный,улица Бородина,44Б
2,Кирпичные,1961-1984,1,среднеэтажка,средний,Советский,мкр. Заозерный,улица Бородина,44Б
3,Кирпичные,1961-1984,мс,малоэтажка,средний,Ленинский,мкр. Порт-Артур,улица 4-я Марьяновская,4А
4,Кирпичные,более2005,1,многоэтажка,первый,Кировский,мкр. 6-й,улица Перелета,12к1
...,...,...,...,...,...,...,...,...,...
2800,Кирпичные,1961-1984,3,малоэтажка,последний,Октябрьский,Крутая Горка мкр,улица Полтавцева,2/2
2801,Кирпичные,1991-2005,3,малоэтажка,средний,Кировский,мкр. Долина нищих,улица 4-я Солнечная,31
2802,Кирпичные,1991-2005,3,малоэтажка,первый,Кировский,мкр. Долина нищих,улица 4-я Солнечная,31
2803,Кирпичные,1991-2005,2,среднеэтажка,средний,Центральный,улица Маршала Жукова,76,NaN


Итоговая таблица

In [ ]:
features = cat_pred + num_pred

In [ ]:
train = omsk[features].dropna()

In [ ]:
train

,material,year_group,type,floortype,floorrange,area,subarea,street,house,price__,build_year,площадь,floor_N,floor_max,rooms,t_,shop_path,cent_path,hish_center_path,mail_path
0,Кирпичные,1961-1984,1,среднеэтажка,последний,Советский,мкр. Город Нефтяников,проспект Мира,82/1,554400.0,1966.0,13.0,5,5,1,1883.0,5.347,8.506,7.171,0.677
1,Кирпичные,1961-1984,1,среднеэтажка,средний,Советский,мкр. Заозерный,улица Бородина,44Б,529848.0,1974.0,13.0,2,5,1,1615.0,1.439,7.351,6.504,0.426
2,Кирпичные,1961-1984,1,среднеэтажка,средний,Советский,мкр. Заозерный,улица Бородина,44Б,554400.0,1974.0,13.0,2,5,1,1884.0,1.439,7.351,6.504,0.426
3,Кирпичные,1961-1984,мс,малоэтажка,средний,Ленинский,мкр. Порт-Артур,улица 4-я Марьяновская,4А,640800.0,1983.0,16.0,2,4,1,1532.0,6.105,7.741,5.810,2.529
4,Кирпичные,более2005,1,многоэтажка,первый,Кировский,мкр. 6-й,улица Перелета,12к1,776160.0,2011.0,16.0,1,9,1,2601.0,7.551,5.491,3.980,0.272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2797,Кирпичные,более2005,3,среднеэтажка,первый,Кировский,мкр. Мельничный,Мельничная улица,89к4,4649040.0,2007.0,120.0,1,5,3,1384.0,3.242,9.675,7.917,2.699
2799,кирпич-бетон,более2005,3,многоэтажка,средний,Кировский,мкр. Прибрежный,улица Перелета,19,5742000.0,2020.0,127.0,10,12,3,2221.0,6.698,4.897,3.426,0.529
2800,Кирпичные,1961-1984,3,малоэтажка,последний,Октябрьский,Крутая Горка мкр,улица Полтавцева,2/2,2016000.0,1979.0,131.0,2,2,3,16.0,2.598,42.831,42.114,2.980
2801,Кирпичные,1991-2005,3,малоэтажка,средний,Кировский,мкр. Долина нищих,улица 4-я Солнечная,31,4824000.0,1991.0,140.9,2,4,3,776.0,8.852,9.951,8.443,0.626
